# Test Develop Mock Circuit

In [1]:
from neuro_dmt.models.bluebrain.circuit.mock.cells import Position, Cell

NameError: name 'WithFields' is not defined

In [ ]:
p = Position(X = 1., Y = 1., Z = 1.)
print(p.as_tuple)
print(p.value)

random_positions =\
    Position.sample(
        (np.array((-1., -1., -1.)), np.array((1., 1., 1.))),
        n = 3)
print(random_positions)

In [ ]:
example_cell =\
    Cell(
        layer=1,
        position=Position(X=1., Y=2., Z=3.),
        mtype="L23_BTC")

In [ ]:
example_cell.as_dict

In [ ]:
from neuro_dmt.models.bluebrain.circuit.mock.test_develop\
    import circuit_composition
circuit_composition.cell_density.head()

In [ ]:
from neuro_dmt.models.bluebrain.circuit.mock import CircuitBuilder

In [ ]:
cb = CircuitBuilder(composition=circuit_composition)

In [ ]:
cb.get_cell_density(mtype="L1_DAC", layer=1)

In [ ]:
print(circuit_composition.cell_density.head())
print(cb.get_number_cells(1, "L1_DAC"))

In [ ]:
circuit_cells = cb.get_cells()

In [ ]:
import pandas as pd
circuit_cells_df =\
    pd.DataFrame([
        cell.as_dict for cell in circuit_cells])
print("number of cells {}".format(circuit_cells_df.shape[0]))
print(circuit_cells_df.head())

In [ ]:
np.random.poisson(1000)